# Auto Identification of Beneficiaries - Data Exploration

**Use Case:** AI-PLATFORM-03 - Auto Identification of Beneficiaries  
**Objective:** Explore eligibility rules, scheme distributions, candidate lists, and ML scoring patterns  
**MLflow Experiment:** `smart/identification_beneficiary/*`

## Overview

This notebook explores:
- Scheme eligibility rules distribution
- Candidate lists and eligibility snapshots
- Rule engine vs ML scorer comparisons
- Scheme-wise eligibility patterns
- Geographic distribution of candidates
- Hybrid evaluator performance
- Prioritization patterns


In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')

# Add paths
project_root = Path().absolute().parent.parent.parent.parent
sys.path.append(str(project_root / 'shared' / 'utils'))
from db_connector import DBConnector

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 7)

# Load config
config_path = Path().absolute().parent.parent / "config" / "db_config.yaml"
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

# Connect to database
db = DBConnector(
    host=config['database']['host'],
    port=config['database']['port'],
    database=config['database']['name'],
    user=config['database']['user'],
    password=config['database']['password']
)
db.connect()

print("✅ Connected to database")
print(f"   Database: {config['database']['name']} at {config['database']['host']}:{config['database']['port']}")
print(f"   Schema: {config['database']['schema']}")


## 1. Scheme & Rules Overview


In [ ]:
# Get schemes with auto-identification enabled
schemes_query = """
SELECT 
    scheme_code,
    scheme_name,
    scheme_type,
    is_auto_id_enabled,
    CASE 
        WHEN scheme_type = 'CASH' THEN 'Cash Transfer'
        WHEN scheme_type = 'NON_CASH' THEN 'Non-Cash Benefit'
        ELSE 'Other'
    END as scheme_category
FROM public.scheme_master
WHERE is_auto_id_enabled = true
ORDER BY scheme_code
"""

schemes_df = pd.read_sql(schemes_query, db.connection)
print(f"📊 Total Schemes with Auto-ID Enabled: {len(schemes_df)}")
print(f"\nScheme Categories:")
print(schemes_df['scheme_category'].value_counts())
print(f"\nFirst 10 Schemes:")
print(schemes_df.head(10).to_string(index=False))


In [ ]:
# Get eligibility rules summary
rules_query = """
SELECT 
    scheme_code,
    COUNT(*) as rule_count,
    COUNT(*) FILTER (WHERE is_mandatory = true) as mandatory_rules,
    COUNT(*) FILTER (WHERE rule_type = 'AGE') as age_rules,
    COUNT(*) FILTER (WHERE rule_type = 'INCOME') as income_rules,
    COUNT(*) FILTER (WHERE rule_type = 'GEOGRAPHY') as geography_rules,
    COUNT(*) FILTER (WHERE rule_type = 'CATEGORY') as category_rules,
    MAX(priority) as max_priority,
    MIN(priority) as min_priority
FROM eligibility.scheme_eligibility_rules
WHERE (effective_to IS NULL OR effective_to >= CURRENT_DATE)
    AND (effective_from <= CURRENT_DATE)
GROUP BY scheme_code
ORDER BY rule_count DESC
"""

rules_summary = pd.read_sql(rules_query, db.connection)
print(f"📋 Rules Summary (Top 15 Schemes):")
print(rules_summary.head(15).to_string(index=False))

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Rule count distribution
axes[0, 0].hist(rules_summary['rule_count'], bins=20, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Distribution of Rules per Scheme')
axes[0, 0].set_xlabel('Number of Rules')
axes[0, 0].set_ylabel('Number of Schemes')

# Mandatory vs Optional rules
top_schemes = rules_summary.head(10)
x = np.arange(len(top_schemes))
width = 0.35
axes[0, 1].bar(x - width/2, top_schemes['mandatory_rules'], width, label='Mandatory', alpha=0.8)
axes[0, 1].bar(x + width/2, top_schemes['rule_count'] - top_schemes['mandatory_rules'], 
               width, label='Optional', alpha=0.8)
axes[0, 1].set_title('Mandatory vs Optional Rules (Top 10 Schemes)')
axes[0, 1].set_xlabel('Scheme')
axes[0, 1].set_ylabel('Number of Rules')
axes[0, 1].set_xticks(x)
axes[0, 1].set_xticklabels(top_schemes['scheme_code'], rotation=45, ha='right')
axes[0, 1].legend()

# Rule types distribution
rule_types = rules_summary[['age_rules', 'income_rules', 'geography_rules', 'category_rules']].sum()
axes[1, 0].pie(rule_types.values, labels=rule_types.index, autopct='%1.1f%%', startangle=90)
axes[1, 0].set_title('Rule Types Distribution Across All Schemes')

# Priority distribution
axes[1, 1].scatter(rules_summary['rule_count'], rules_summary['max_priority'], 
                   alpha=0.6, s=100)
axes[1, 1].set_title('Max Priority vs Number of Rules')
axes[1, 1].set_xlabel('Number of Rules')
axes[1, 1].set_ylabel('Maximum Priority')

plt.tight_layout()
plt.show()


20 

In [ ]:
# Check if eligibility_snapshots table exists and has data
try:
    snapshots_query = """
    SELECT 
        snapshot_id,
        snapshot_date,
        scheme_code,
        COUNT(*) FILTER (WHERE rule_status = 'ELIGIBLE') as rule_eligible,
        COUNT(*) FILTER (WHERE rule_status = 'NOT_ELIGIBLE') as rule_not_eligible,
        COUNT(*) FILTER (WHERE rule_status = 'POSSIBLE_ELIGIBLE') as rule_possible,
        AVG(ml_score) as avg_ml_score,
        AVG(hybrid_score) as avg_hybrid_score,
        COUNT(*) as total_candidates
    FROM eligibility.eligibility_snapshots
    GROUP BY snapshot_id, snapshot_date, scheme_code
    ORDER BY snapshot_date DESC
    LIMIT 100
    """
    snapshots_df = pd.read_sql(snapshots_query, db.connection)
    
    if len(snapshots_df) > 0:
        print(f"📸 Found {len(snapshots_df)} snapshot records")
        print("\nSnapshot Summary:")
        print(snapshots_df.describe())
        
        # Visualization
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        
        # Eligibility status distribution
        status_counts = snapshots_df[['rule_eligible', 'rule_not_eligible', 'rule_possible']].sum()
        axes[0, 0].bar(status_counts.index, status_counts.values, color=['green', 'red', 'orange'], alpha=0.7)
        axes[0, 0].set_title('Overall Eligibility Status Distribution')
        axes[0, 0].set_ylabel('Count')
        axes[0, 0].tick_params(axis='x', rotation=45)
        
        # ML Score vs Hybrid Score
        if snapshots_df['avg_ml_score'].notna().sum() > 0:
            axes[0, 1].scatter(snapshots_df['avg_ml_score'], snapshots_df['avg_hybrid_score'], 
                             alpha=0.6, s=50)
            axes[0, 1].plot([0, 1], [0, 1], 'r--', alpha=0.5)
            axes[0, 1].set_title('ML Score vs Hybrid Score')
            axes[0, 1].set_xlabel('Average ML Score')
            axes[0, 1].set_ylabel('Average Hybrid Score')
        
        # Candidates per scheme
        scheme_counts = snapshots_df.groupby('scheme_code')['total_candidates'].sum().sort_values(ascending=False).head(10)
        axes[1, 0].barh(range(len(scheme_counts)), scheme_counts.values, alpha=0.7)
        axes[1, 0].set_yticks(range(len(scheme_counts)))
        axes[1, 0].set_yticklabels(scheme_counts.index)
        axes[1, 0].set_title('Top 10 Schemes by Total Candidates')
        axes[1, 0].set_xlabel('Total Candidates')
        
        # Timeline of snapshots
        if snapshots_df['snapshot_date'].notna().sum() > 0:
            timeline = snapshots_df.groupby(snapshots_df['snapshot_date'].dt.date)['total_candidates'].sum()
            axes[1, 1].plot(timeline.index, timeline.values, marker='o', alpha=0.7)
            axes[1, 1].set_title('Candidates Over Time')
            axes[1, 1].set_xlabel('Date')
            axes[1, 1].set_ylabel('Total Candidates')
            axes[1, 1].tick_params(axis='x', rotation=45)
        
        plt.tight_layout()
        plt.show()
    else:
        print("⚠️ No snapshot data found. This is expected if no evaluations have been run yet.")
except Exception as e:
    print(f"⚠️ Eligibility snapshots table may not exist or have no data: {e}")
    print("   This is expected if no evaluations have been run yet.")


In [ ]:
# Check candidate lists
try:
    candidates_query = """
    SELECT 
        list_id,
        list_name,
        scheme_code,
        list_type,
        priority_cutoff,
        total_candidates,
        created_at
    FROM eligibility.candidate_lists
    ORDER BY created_at DESC
    LIMIT 50
    """
    candidates_df = pd.read_sql(candidates_query, db.connection)
    
    if len(candidates_df) > 0:
        print(f"📋 Found {len(candidates_df)} candidate lists")
        print("\nCandidate Lists Summary:")
        print(candidates_df.describe(include='all'))
        
        # Visualization
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        
        # List type distribution
        list_types = candidates_df['list_type'].value_counts()
        axes[0, 0].pie(list_types.values, labels=list_types.index, autopct='%1.1f%%', startangle=90)
        axes[0, 0].set_title('Candidate List Types Distribution')
        
        # Candidates per scheme
        scheme_candidates = candidates_df.groupby('scheme_code')['total_candidates'].sum().sort_values(ascending=False).head(10)
        axes[0, 1].barh(range(len(scheme_candidates)), scheme_candidates.values, alpha=0.7)
        axes[0, 1].set_yticks(range(len(scheme_candidates)))
        axes[0, 1].set_yticklabels(scheme_candidates.index)
        axes[0, 1].set_title('Top 10 Schemes by Total Candidates in Lists')
        axes[0, 1].set_xlabel('Total Candidates')
        
        # Priority cutoff distribution
        if candidates_df['priority_cutoff'].notna().sum() > 0:
            axes[1, 0].hist(candidates_df['priority_cutoff'].dropna(), bins=20, edgecolor='black', alpha=0.7)
            axes[1, 0].set_title('Priority Cutoff Distribution')
            axes[1, 0].set_xlabel('Priority Cutoff')
            axes[1, 0].set_ylabel('Frequency')
        
        # Lists over time
        if candidates_df['created_at'].notna().sum() > 0:
            timeline = candidates_df.groupby(candidates_df['created_at'].dt.date)['total_candidates'].sum()
            axes[1, 1].plot(timeline.index, timeline.values, marker='o', alpha=0.7)
            axes[1, 1].set_title('Candidate Lists Creation Over Time')
            axes[1, 1].set_xlabel('Date')
            axes[1, 1].set_ylabel('Total Candidates')
            axes[1, 1].tick_params(axis='x', rotation=45)
        
        plt.tight_layout()
        plt.show()
    else:
        print("⚠️ No candidate lists found. This is expected if no evaluations have been run yet.")
except Exception as e:
    print(f"⚠️ Candidate lists table may not exist or have no data: {e}")
    print("   This is expected if no evaluations have been run yet.")


## 4. Rule Engine vs ML Scorer Comparison


In [ ]:
# Compare rule engine and ML scorer outputs
try:
    comparison_query = """
    SELECT 
        scheme_code,
        rule_status,
        COUNT(*) as count,
        AVG(ml_score) as avg_ml_score,
        AVG(hybrid_score) as avg_hybrid_score,
        STDDEV(ml_score) as std_ml_score,
        STDDEV(hybrid_score) as std_hybrid_score
    FROM eligibility.eligibility_snapshots
    WHERE ml_score IS NOT NULL
    GROUP BY scheme_code, rule_status
    ORDER BY scheme_code, rule_status
    """
    comparison_df = pd.read_sql(comparison_query, db.connection)
    
    if len(comparison_df) > 0:
        print("📊 Rule Engine vs ML Scorer Comparison:")
        print(comparison_df.head(20).to_string(index=False))
        
        # Visualization
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        
        # Rule status distribution
        status_dist = comparison_df.groupby('rule_status')['count'].sum()
        axes[0, 0].bar(status_dist.index, status_dist.values, 
                      color=['green', 'red', 'orange'], alpha=0.7)
        axes[0, 0].set_title('Rule Engine Status Distribution')
        axes[0, 0].set_ylabel('Count')
        axes[0, 0].tick_params(axis='x', rotation=45)
        
        # ML scores by rule status
        for status in comparison_df['rule_status'].unique():
            subset = comparison_df[comparison_df['rule_status'] == status]
            if len(subset) > 0:
                axes[0, 1].bar(subset['scheme_code'].head(10), subset['avg_ml_score'].head(10), 
                              label=status, alpha=0.7)
        axes[0, 1].set_title('Average ML Score by Rule Status (Top 10 Schemes)')
        axes[0, 1].set_xlabel('Scheme Code')
        axes[0, 1].set_ylabel('Average ML Score')
        axes[0, 1].tick_params(axis='x', rotation=45)
        axes[0, 1].legend()
        
        # Hybrid scores by rule status
        for status in comparison_df['rule_status'].unique():
            subset = comparison_df[comparison_df['rule_status'] == status]
            if len(subset) > 0:
                axes[1, 0].bar(subset['scheme_code'].head(10), subset['avg_hybrid_score'].head(10), 
                              label=status, alpha=0.7)
        axes[1, 0].set_title('Average Hybrid Score by Rule Status (Top 10 Schemes)')
        axes[1, 0].set_xlabel('Scheme Code')
        axes[1, 0].set_ylabel('Average Hybrid Score')
        axes[1, 0].tick_params(axis='x', rotation=45)
        axes[1, 0].legend()
        
        # Score comparison
        eligible = comparison_df[comparison_df['rule_status'] == 'ELIGIBLE']
        if len(eligible) > 0:
            axes[1, 1].scatter(eligible['avg_ml_score'], eligible['avg_hybrid_score'], 
                             alpha=0.6, s=100, label='ELIGIBLE')
        not_eligible = comparison_df[comparison_df['rule_status'] == 'NOT_ELIGIBLE']
        if len(not_eligible) > 0:
            axes[1, 1].scatter(not_eligible['avg_ml_score'], not_eligible['avg_hybrid_score'], 
                             alpha=0.6, s=100, label='NOT_ELIGIBLE', color='red')
        axes[1, 1].plot([0, 1], [0, 1], 'k--', alpha=0.3)
        axes[1, 1].set_title('ML Score vs Hybrid Score by Rule Status')
        axes[1, 1].set_xlabel('Average ML Score')
        axes[1, 1].set_ylabel('Average Hybrid Score')
        axes[1, 1].legend()
        
        plt.tight_layout()
        plt.show()
    else:
        print("⚠️ No comparison data found. This is expected if no evaluations have been run yet.")
except Exception as e:
    print(f"⚠️ Comparison data may not be available: {e}")
    print("   This is expected if no evaluations have been run yet.")


required

In [ ]:
# Analyze geographic distribution (if available in snapshots)
try:
    geo_query = """
    SELECT 
        district_id,
        block_id,
        COUNT(*) as candidate_count,
        AVG(hybrid_score) as avg_score,
        COUNT(*) FILTER (WHERE rule_status = 'ELIGIBLE') as eligible_count
    FROM eligibility.eligibility_snapshots
    WHERE district_id IS NOT NULL
    GROUP BY district_id, block_id
    ORDER BY candidate_count DESC
    LIMIT 50
    """
    geo_df = pd.read_sql(geo_query, db.connection)
    
    if len(geo_df) > 0:
        print("🗺️ Geographic Distribution:")
        print(geo_df.head(20).to_string(index=False))
        
        # Visualization
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        
        # Top districts by candidates
        district_counts = geo_df.groupby('district_id')['candidate_count'].sum().sort_values(ascending=False).head(10)
        axes[0, 0].barh(range(len(district_counts)), district_counts.values, alpha=0.7)
        axes[0, 0].set_yticks(range(len(district_counts)))
        axes[0, 0].set_yticklabels(district_counts.index)
        axes[0, 0].set_title('Top 10 Districts by Candidate Count')
        axes[0, 0].set_xlabel('Candidate Count')
        
        # Average score by district
        district_scores = geo_df.groupby('district_id')['avg_score'].mean().sort_values(ascending=False).head(10)
        axes[0, 1].barh(range(len(district_scores)), district_scores.values, alpha=0.7, color='green')
        axes[0, 1].set_yticks(range(len(district_scores)))
        axes[0, 1].set_yticklabels(district_scores.index)
        axes[0, 1].set_title('Top 10 Districts by Average Score')
        axes[0, 1].set_xlabel('Average Score')
        
        # Eligibility rate by district
        district_eligibility = geo_df.groupby('district_id').agg({
            'candidate_count': 'sum',
            'eligible_count': 'sum'
        })
        district_eligibility['eligibility_rate'] = district_eligibility['eligible_count'] / district_eligibility['candidate_count']
        top_eligibility = district_eligibility.nlargest(10, 'eligibility_rate')
        axes[1, 0].barh(range(len(top_eligibility)), top_eligibility['eligibility_rate'].values, 
                       alpha=0.7, color='orange')
        axes[1, 0].set_yticks(range(len(top_eligibility)))
        axes[1, 0].set_yticklabels(top_eligibility.index)
        axes[1, 0].set_title('Top 10 Districts by Eligibility Rate')
        axes[1, 0].set_xlabel('Eligibility Rate')
        
        # Scatter: candidates vs eligibility rate
        axes[1, 1].scatter(district_eligibility['candidate_count'], 
                          district_eligibility['eligibility_rate'], 
                          alpha=0.6, s=100)
        axes[1, 1].set_title('Candidate Count vs Eligibility Rate by District')
        axes[1, 1].set_xlabel('Candidate Count')
        axes[1, 1].set_ylabel('Eligibility Rate')
        
        plt.tight_layout()
        plt.show()
    else:
        print("⚠️ No geographic data found. This is expected if no evaluations have been run yet.")
except Exception as e:
    print(f"⚠️ Geographic data may not be available: {e}")
    print("   This is expected if no evaluations have been run yet.")


## 6. Summary & Insights


In [ ]:
print("="*80)
print("📊 DATA EXPLORATION SUMMARY")
print("="*80)
print(f"\n✅ Schemes with Auto-ID: {len(schemes_df)}")
print(f"✅ Total Rules: {rules_summary['rule_count'].sum() if len(rules_summary) > 0 else 0}")

try:
    if 'snapshots_df' in locals() and len(snapshots_df) > 0:
        print(f"✅ Eligibility Snapshots: {len(snapshots_df)}")
        print(f"   - Total Candidates: {snapshots_df['total_candidates'].sum():,.0f}")
        print(f"   - Average ML Score: {snapshots_df['avg_ml_score'].mean():.3f}")
        print(f"   - Average Hybrid Score: {snapshots_df['avg_hybrid_score'].mean():.3f}")
except:
    print("⚠️  No snapshot data available")

try:
    if 'candidates_df' in locals() and len(candidates_df) > 0:
        print(f"✅ Candidate Lists: {len(candidates_df)}")
        print(f"   - Total Candidates in Lists: {candidates_df['total_candidates'].sum():,.0f}")
except:
    print("⚠️  No candidate list data available")

print("\n" + "="*80)
print("💡 Key Insights:")
print("="*80)
print("1. Review rule distributions to identify schemes with complex eligibility criteria")
print("2. Compare ML scores with rule engine outputs to validate hybrid evaluator")
print("3. Analyze geographic patterns to identify coverage gaps")
print("4. Monitor candidate list generation patterns for prioritization insights")
print("\n🔍 Next Steps:")
print("- Run fairness audit notebook to check for demographic biases")
print("- Execute model training if sufficient historical data is available")
print("- Review prioritization patterns for optimization opportunities")


In [ ]:
# Close database connection
db.disconnect()
print("✅ Database connection closed")
